<a href="https://colab.research.google.com/github/beyzaturku/image_processing_in_autonomous_vehicles/blob/main/car_detect_and_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
!pip install deep-sort-realtime
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
!pip install deep-sort-realtime

In [3]:
import os
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
from IPython.display import display, HTML
import matplotlib.pyplot as plt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
print("Lütfen bir test videosu yükleyin:")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

Lütfen bir test videosu yükleyin:


Saving sonuc_video (4).mp4 to sonuc_video (4).mp4


In [5]:
print("YOLOv8 modeli yükleniyor...")
model = YOLO("yolov8m.pt")

YOLOv8 modeli yükleniyor...


100%|██████████| 49.7M/49.7M [00:00<00:00, 502MB/s]


In [11]:
# Araç sınıfları
arac_siniflari = [2, 3, 5, 7]  # 2:araba, 3:motosiklet, 5:otobüs, 7:kamyon
sinif_isimleri = {2: "Araba", 3: "Motosiklet", 5: "Otobus", 7: "Kamyon"}

In [7]:
# DeepSORT Takipçisini Başlatma
tracker = DeepSort(max_age=30)

In [8]:
np.random.seed(42)  # Tutarlı renkler için
COLORS = np.random.randint(0, 255, size=(100, 3), dtype="uint8")

In [9]:
def process_video(input_path, output_path, box_scale=0.8):
    cap = cv2.VideoCapture(input_path)

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_count = 0
    arac_sayilari = {2: 0, 3: 0, 5: 0, 7: 0}  # Sınıf başına araç sayısı
    aktif_araclar = {2: 0, 3: 0, 5: 0, 7: 0}  # Anlık aktif araç sayısı

    track_history = {}
    track_speed = {}

    prev_time = cv2.getTickCount()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # FPS hesapla
        curr_time = cv2.getTickCount()
        time_diff = (curr_time - prev_time) / cv2.getTickFrequency()
        fps_calc = 1.0 / time_diff
        prev_time = curr_time

        # YOLOv8 ile araç tespiti yap
        results = model(frame)

        detections = []

        for result in results:
            boxes = result.boxes.cpu().numpy()

            for box in boxes:
                class_id = int(box.cls[0])
                if class_id in arac_siniflari:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = float(box.conf[0])

                    if confidence > 0.5:
                        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                        box_width = int((x2 - x1) * box_scale)
                        box_height = int((y2 - y1) * box_scale)

                        x1_new = max(0, center_x - box_width // 2)
                        y1_new = max(0, center_y - box_height // 2)
                        x2_new = min(width, center_x + box_width // 2)
                        y2_new = min(height, center_y + box_height // 2)

                        detection = [[x1_new, y1_new, x2_new - x1_new, y2_new - y1_new], confidence, class_id]
                        detections.append(detection)

        # DeepSORT Takip Algoritması
        if detections:
            tracks = tracker.update_tracks(detections, frame=frame)

            for cls_id in aktif_araclar:
                aktif_araclar[cls_id] = 0

            for track in tracks:
                if not track.is_confirmed():
                    continue

                ltrb = track.to_ltrb()
                x1, y1, x2, y2 = map(int, ltrb)
                track_id = track.track_id

                if isinstance(track_id, str):
                    track_id = int(track_id)

                class_id = None
                if hasattr(track, 'det_class'):
                    class_id = track.det_class
                elif hasattr(track, 'class_id'):
                    class_id = track.class_id

                color = [int(c) for c in COLORS[track_id % len(COLORS)]]

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                center_point = (center_x, center_y)

                if track_id not in track_history:
                    track_history[track_id] = []

                track_history[track_id].append(center_point)
                if len(track_history[track_id]) > 20:
                    track_history[track_id].pop(0)

                if len(track_history[track_id]) > 1:
                    for i in range(1, len(track_history[track_id])):
                        pt1 = track_history[track_id][i-1]
                        pt2 = track_history[track_id][i]
                        cv2.line(frame, pt1, pt2, color, 2)

                if track_id in track_speed:
                    last_pos, last_time = track_speed[track_id]

                    dist = np.sqrt((center_x - last_pos[0])**2 + (center_y - last_pos[1])**2)
                    time_elapsed = frame_count - last_time

                    speed = dist / time_elapsed * fps * 0.1

                    cv2.putText(frame, f"{speed:.1f} km/s", (x1, y1 - 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                track_speed[track_id] = (center_point, frame_count)

                if track.is_confirmed() and track.time_since_update == 0:
                    if class_id in arac_siniflari:
                        arac_sayilari[class_id] += 1
                        aktif_araclar[class_id] += 1

                if class_id is not None and class_id in arac_siniflari:
                    text = f"ID: {track_id} - {sinif_isimleri[class_id]}"
                else:
                    text = f"ID: {track_id}"

                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        cv2.putText(frame, f"FPS: {fps_calc:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        cv2.putText(frame, f"Kare: {frame_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        y_pos = 30
        cv2.putText(frame, "Anlik Araclar:", (width - 200, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        y_pos += 30

        for class_id in arac_siniflari:
            sinif_text = f"{sinif_isimleri[class_id]}: {aktif_araclar[class_id]}"
            cv2.putText(frame, sinif_text, (width - 200, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            y_pos += 30

        y_pos = height - 150
        cv2.putText(frame, "Toplam Araclar:", (width - 200, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        y_pos += 30

        for class_id in arac_siniflari:
            sinif_text = f"{sinif_isimleri[class_id]}: {arac_sayilari[class_id]}"
            cv2.putText(frame, sinif_text, (width - 200, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            y_pos += 30

        out.write(frame)

    print(f"Toplam islenen kare sayisi: {frame_count}")
    print(f"Tespit edilen arac sayilari:")
    for class_id, count in arac_sayilari.items():
        print(f"  {sinif_isimleri[class_id]}: {count}")

    # Temizleme
    cap.release()
    out.release()
    return arac_sayilari

In [12]:
output_path = "sonuc_video.mp4"
print("Video işleniyor...")

arac_sayilari = process_video(video_path, output_path, box_scale=0.8)

print("İşlenmiş video indiriliyor...")
files.download(output_path)

print("İşlem tamamlandı!")

Video işleniyor...

0: 384x640 1 potted plant, 12.4ms
Speed: 3.4ms preprocess, 12.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.1ms
Speed: 2.0ms preprocess, 10.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.9ms
Speed: 1.9ms preprocess, 9.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.8ms
Speed: 1.9ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.9ms
Speed: 1.9ms preprocess, 9.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.8ms
Speed: 1.9ms preprocess, 9.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.2ms
Speed: 2.5ms preprocess, 11.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 9.9ms
Speed: 1.9ms preprocess,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

İşlem tamamlandı!
